<a href="https://colab.research.google.com/github/abelalanoca/P.MAGNO/blob/main/TP1/B_Y_ER_optimizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files, drive
import pandas as pd
import itertools
import re

# 1️⃣ Subir archivo desde tu PC
uploaded = files.upload()

# 2️⃣ Montar Google Drive (si quieres guardar/leer desde ahí)
drive.mount("/content/drive")

# 3️⃣ Leer hoja 1 del archivo subido
nombre_archivo = list(uploaded.keys())[0]
df_raw = pd.read_excel(nombre_archivo, sheet_name=0)

# --------------------------------------------------------
# 🔹 Función: Normalizar nombres de bancos
# --------------------------------------------------------
def reemplazar_bancos_especificos(df):
    df_1 = df.copy()
    reemplazos = {
        "COMPARTAMOS BANCO": "Compartamos Banco*",
        "BANCO DE COMERCIO": "BANCOM"
    }
    df_1 = df_1.applymap(
        lambda x: reemplazos.get(str(x).strip().upper(), x) if isinstance(x, str) else x
    )
    return df_1

df_1 = reemplazar_bancos_especificos(df_raw)

# --------------------------------------------------------
# 🔹 Función: Extraer listas de cuentas y bancos
# --------------------------------------------------------
def extraer_listas_balance(df):
    primera_col = df.iloc[:, 0].astype(str).str.strip().str.upper()

    idx_activo = primera_col[primera_col == "ACTIVO"].index[0]
    idx_total_activo = primera_col[primera_col == "TOTAL ACTIVO"].index[0]

    cuentas_del_activo = (
        df.iloc[idx_activo:idx_total_activo + 1, 0]
        .dropna().astype(str).str.strip().tolist()
    )
    cuentas_del_activo = [c for c in cuentas_del_activo if c.upper() != "ACTIVO"]

    idx_pasivo = primera_col[primera_col == "PASIVO"].index[0]
    idx_otras_conting = primera_col[primera_col == "OTRAS CUENTAS CONTINGENTES"].index[0]

    lista_pasivos = (
        df.iloc[idx_pasivo:idx_otras_conting + 1, 0]
        .dropna().astype(str).str.strip().tolist()
    )
    lista_pasivos = [p for p in lista_pasivos if p.upper() != "PASIVO"]

    bancos_activo = (
        df.iloc[idx_activo, 1:]
        .dropna().astype(str).str.strip().tolist()
    )
    bancos_activo = [b for b in bancos_activo if b.upper() != "ACTIVO"]

    return cuentas_del_activo, lista_pasivos, bancos_activo

cuentas_del_activo, lista_pasivos, bancos_activo = extraer_listas_balance(df_1)

# --------------------------------------------------------
# 🔹 Función: Extraer fecha
# --------------------------------------------------------
def extraer_fecha(df):
    idx_balance = df[df.astype(str).apply(
        lambda row: row.str.contains("Balance General por Empresa Bancaria", case=False, na=False)
    ).any(axis=1)].index[0]

    fecha_cruda = df.iloc[idx_balance + 1].dropna().astype(str).tolist()[0]
    fecha_dt = pd.to_datetime(fecha_cruda, errors="coerce")
    return [fecha_dt.strftime("%b-%d") if pd.notnull(fecha_dt) else fecha_cruda]

Fecha = extraer_fecha(df_1)

# --------------------------------------------------------
# 🔹 Función: Crear estructura vacía df_2
# --------------------------------------------------------
def crear_df_2(Fecha, bancos_activo, cuentas_del_activo, lista_pasivos):
    def limpiar_lista(lista):
        return [x for x in lista if pd.notnull(x) and str(x).strip() != ""]

    cuentas_del_activo = limpiar_lista(cuentas_del_activo)
    lista_pasivos = limpiar_lista(lista_pasivos)
    bancos_activo = limpiar_lista(bancos_activo)

    columnas_cuentas = cuentas_del_activo + lista_pasivos
    df_2 = pd.DataFrame(columns=["MES", "ENTIDAD", "MONEDA"] + columnas_cuentas)

    entidades, monedas, meses = [], [], []
    for banco in bancos_activo:
        for moneda in ["MN", "ME", "TOTAL"]:
            entidades.append(banco)
            monedas.append(moneda)
            meses.append(Fecha[0])

    df_2["MES"] = meses
    df_2["ENTIDAD"] = entidades
    df_2["MONEDA"] = monedas

    return df_2

df_2 = crear_df_2(Fecha, bancos_activo, cuentas_del_activo, lista_pasivos)

# --------------------------------------------------------
# 🔹 Función: Llenar ACTIVO o PASIVO en df_2
# --------------------------------------------------------
def llenar_df_2_matriz(df_1, df_2, bancos, cuentas, tipo="ACTIVO"):
    df_limpio = df_1.copy().applymap(lambda x: str(x).strip() if pd.notnull(x) else x)
    idx_tipo = df_limpio[df_limpio.iloc[:, 0].str.upper() == tipo].index[0]

    filas_cuentas = {}
    for cuenta in cuentas:
        match = df_limpio.index[df_limpio.iloc[:, 0].str.upper() == cuenta.upper()].tolist()
        if match:
            filas_cuentas[cuenta] = match[0]

    columnas_bancos = {}
    for banco in bancos:
        match = df_limpio.columns[df_limpio.iloc[idx_tipo, :].astype(str) == banco].tolist()
        if match:
            columnas_bancos[banco] = df_limpio.columns.get_loc(match[0])

    for banco, col_idx in columnas_bancos.items():
        for cuenta, fila_idx in filas_cuentas.items():
            valores = df_1.iloc[fila_idx, col_idx:col_idx+3].tolist()
            for moneda, valor in zip(["MN", "ME", "TOTAL"], valores):
                df_2.loc[(df_2["ENTIDAD"] == banco) & (df_2["MONEDA"] == moneda), cuenta] = valor

    return df_2

df_2 = llenar_df_2_matriz(df_1, df_2, bancos_activo, cuentas_del_activo, tipo="ACTIVO")
df_2 = llenar_df_2_matriz(df_1, df_2, bancos_activo, lista_pasivos, tipo="PASIVO")

# limpiar encabezados
df_2.columns = df_2.columns.map(lambda x: ''.join(ch for ch in str(x) if ch.isalpha() or ch.isspace()))

# --------------------------------------------------------
# 🔹 Leer hoja 2 y crear df_3 (tu misma lógica)
# --------------------------------------------------------
df_raw_hoja2 = pd.read_excel(nombre_archivo, sheet_name=1)

primera_col_hoja2 = df_raw_hoja2.iloc[:, 0].astype(str).str.strip()
idx_ingresos = primera_col_hoja2[primera_col_hoja2.str.contains("INGRESOS FINANCIEROS", case=False, na=False)].index[0]
idx_resultado = primera_col_hoja2[primera_col_hoja2.str.contains("RESULTADO NETO DEL EJERCICIO", case=False, na=False)].index[0]

Cuentas_del_ER = df_raw_hoja2.iloc[idx_ingresos:idx_resultado + 1, 0].dropna().astype(str).str.strip().tolist()

mask_bbva = df_raw_hoja2.apply(lambda row: row.astype(str).str.contains("Banco BBVA Perú", case=False, na=False).any(), axis=1)
mask_total = df_raw_hoja2.apply(lambda row: row.astype(str).str.contains("Total Banca Múltiple", case=False, na=False).any(), axis=1)

idx_bbva = mask_bbva[mask_bbva].index[0]
idx_total = mask_total[mask_total].index[0]

Bancos_del_ER = []
for i in range(idx_bbva, idx_total + 1):
    fila = df_raw_hoja2.iloc[i, :].dropna().astype(str).str.strip().tolist()
    Bancos_del_ER.extend(fila)

columnas_ER = ["MES", "ENTIDAD", "MONEDA"] + Cuentas_del_ER
df_3 = pd.DataFrame(columns=columnas_ER)

entidades = list(itertools.chain.from_iterable([[banco]*3 for banco in Bancos_del_ER]))
monedas = ["MN", "ME", "TOTAL"] * len(Bancos_del_ER)
meses = [Fecha[0]] * len(entidades)

df_3["ENTIDAD"] = entidades
df_3["MONEDA"] = monedas
df_3["MES"] = meses

def normalizar(texto):
    texto = str(texto).upper()
    texto = re.sub(r"[ÁÀÄÂ]", "A", texto)
    texto = re.sub(r"[ÉÈËÊ]", "E", texto)
    texto = re.sub(r"[ÍÌÏÎ]", "I", texto)
    texto = re.sub(r"[ÓÒÖÔ]", "O", texto)
    texto = re.sub(r"[ÚÙÜÛ]", "U", texto)
    texto = re.sub(r"[\(\)]", "", texto)
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

df_limpio = df_raw_hoja2.copy().astype(str).apply(lambda col: col.str.strip())
df_limpio.iloc[:, 0] = df_limpio.iloc[:, 0].map(normalizar)

for banco in Bancos_del_ER:
    banco_norm = normalizar(banco)
    mask_fila_banco = df_limpio.apply(lambda row: row.astype(str).map(normalizar).str.contains(banco_norm).any(), axis=1)
    if not mask_fila_banco.any():
        continue
    fila_banco_idx = mask_fila_banco[mask_fila_banco].index[0]

    for cuenta in Cuentas_del_ER:
        cuenta_norm = normalizar(cuenta)
        mask_cuenta = df_limpio.iloc[:, 0].str.contains(cuenta_norm)
        if not mask_cuenta.any():
            continue
        fila_cuenta_idx = mask_cuenta[mask_cuenta].index[0]

        mask_col = df_limpio.iloc[fila_banco_idx, :].map(normalizar).str.contains(banco_norm)
        if not mask_col.any():
            continue
        col_banco_idx = df_limpio.columns.get_loc(mask_col[mask_col].index[0])

        valores = df_raw_hoja2.iloc[fila_cuenta_idx, col_banco_idx:col_banco_idx+3].tolist()

        for moneda, valor in zip(["MN", "ME", "TOTAL"], valores):
            df_3.loc[(df_3["ENTIDAD"] == banco) & (df_3["MONEDA"] == moneda), cuenta] = valor

# --------------------------------------------------------
# 🔹 Enviar df_2 y df_3 a Google Sheets
# --------------------------------------------------------
!pip install --quiet gspread gspread_dataframe
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()
client = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1E4pZi2ccNbycPfPBpGchqDiw8o2wtJHqHGSR9rN0dnI/edit#gid=0"
spreadsheet = client.open_by_url(spreadsheet_url)

# df_2 → hoja 1
try:
    worksheet1 = spreadsheet.worksheet("df_2")
except:
    worksheet1 = spreadsheet.add_worksheet(title="df_2", rows=1000, cols=1000)
worksheet1.clear()
set_with_dataframe(worksheet1, df_2)

# df_3 → hoja 2
try:
    worksheet2 = spreadsheet.worksheet("df_3")
except:
    worksheet2 = spreadsheet.add_worksheet(title="df_3", rows=1000, cols=1000)
worksheet2.clear()
set_with_dataframe(worksheet2, df_3)

print("✅ df_2 y df_3 enviados a Google Sheets")


Saving B-2201-my2025.XLS to B-2201-my2025 (2).XLS
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-4262515314.py:25: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_1 = df_1.applymap(
/tmp/ipython-input-4262515314.py:113: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_1.copy().applymap(lambda x: str(x).strip() if pd.notnull(x) else x)
/tmp/ipython-input-4262515314.py:113: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_1.copy().applymap(lambda x: str(x).strip() if pd.notnull(x) else x)


✅ df_2 y df_3 enviados a Google Sheets


In [6]:
print(bancos_activo)

['Banco BBVA Perú', 'BANCOM', 'Banco de Crédito del Perú', 'Banco Pichincha', 'Banco Interamericano de Finanzas', 'Scotiabank Perú', 'Citibank', 'Interbank', 'Mibanco', 'Banco GNB', 'Banco Falabella Perú', 'Banco Santander Perú', 'Banco Ripley', 'Alfin Banco', 'Banco ICBC', 'Bank of China', 'Banco BCI Perú', 'Compartamos Banco*', 'Total Banca Múltiple', 'Banco de Crédito con Sucursales en el \nExterior', 'Total Banca Múltiple Incluye Sucursales en el Exterior']
